# QrackCircuit

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

In [1]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install pyqrack

[`QrackSimulator`](https://github.com/vm6502q/pyqrack/blob/main/pyqrack/qrack_simulator.py) is the "workhorse" of the `pyqrack` package. It instantiates simulated "registers" of qubits that we can act basic quantum gates between, to form arbitrary universal quantum circuits.

`QrackCircuit` is an optional class for optimizing compilation. With it, one can define a circuit in advance, which is optimized upon definition. Then, one can save the optimized result to a file and later load it into a new or existing `QrackCircuit` instance. Ultimately, the circuit is executed by calling `run()` on a `QrackCircuit`, with a parameter of `QrackSimulator` of appropriate size. (The necessary width of the `QrackSimulator` can be determined with `QrackCircuit.get_qubit_count()`.)

In [2]:
import math
import random

sqrt1_2 = 1 / math.sqrt(2)

def x_to_y(circ, q):
    circ.mtrx([1, 0, 0, 1j], q)
    return 1

def x_to_z(circ, q):
    circ.mtrx([sqrt1_2, sqrt1_2, sqrt1_2, -sqrt1_2], q)
    return 1

def y_to_z(circ, q):
    circ.mtrx([1, 0, 0, -1j], q)
    circ.mtrx([sqrt1_2, sqrt1_2, sqrt1_2, -sqrt1_2], q)
    return 2

def y_to_x(circ, q):
    circ.mtrx([1, 0, 0, -1j], q)
    return 1

def z_to_x(circ, q):
    circ.mtrx([sqrt1_2, sqrt1_2, sqrt1_2, -sqrt1_2], q)
    return 1

def z_to_y(circ, q):
    circ.mtrx([sqrt1_2, sqrt1_2, sqrt1_2, -sqrt1_2], q)
    circ.mtrx([1, 0, 0, 1j], q)
    return 2

def cx(circ, q1, q2):
    circ.ucmtrx([q1], [0, 1, 1, 0], q2, 1)
    return 1

def cy(circ, q1, q2):
    circ.ucmtrx([q1], [0, -1j, 1j, 0], q2, 1)
    return 1

def cz(circ, q1, q2):
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 1)
    return 1

def acx(circ, q1, q2):
    circ.ucmtrx([q1], [0, 1, 1, 0], q2, 0)
    return 1

def acy(circ, q1, q2):
    circ.ucmtrx([q1], [0, -1j, 1j, 0], q2, 0)
    return 1

def acz(circ, q1, q2):
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 0)
    return 1

def swap(circ, q1, q2):
    circ.swap(q1, q2)
    return 1

def ident(circ, q1, q2):
    return 0

def random_circuit(width, circ):
    gate_count = 0
    num_qubits = width
    single_bit_gates = x_to_y, x_to_z, y_to_z, y_to_x, z_to_x, z_to_y
    # two_bit_gates = ident, ident, cx, cz, cy, acx, acz, acy
    two_bit_gates = swap, ident, cx, cz, cy, acx, acz, acy
    gateSequence = [ 0, 3, 2, 1, 2, 1, 0, 3 ]
    colLen = math.floor(math.sqrt(num_qubits))
    while ((math.floor(num_qubits / colLen) * colLen) != num_qubits):
        colLen = colLen - 1
    rowLen = num_qubits // colLen;

    for i in range(4 * width):
        # Single bit gates
        for j in range(num_qubits):
            # Random basis switch
            gate = random.choice(single_bit_gates)
            gate_count += gate(circ, j)
            rnd = random.uniform(0, 4 * math.pi)
            circ.mtrx([1, 0, 0, math.cos(rnd) + math.sin(rnd) * 1j], j)
            ++gate_count

        gate = gateSequence[0]
        gateSequence.pop(0)
        gateSequence.append(gate)

        for row in range(1, rowLen, 2):
            for col in range(0, colLen):
                tempRow = row;
                tempCol = col;

                tempRow = tempRow + (1 if (gate & 2) else -1)
                if colLen != 1:
                    tempCol = tempCol + (1 if (gate & 1) else 0)

                if (tempRow < 0) or (tempCol < 0) or (tempRow >= rowLen) or (tempCol >= colLen):
                    continue;

                b1 = row * colLen + col;
                b2 = tempRow * colLen + tempCol;

                # Two bit gates
                g = random.choice(two_bit_gates)
                gate_count += g(circ, b1, b2)

    print("Gate count (before optimization): " + str(gate_count))

    return circ

In [3]:
from pyqrack import QrackSimulator, QrackCircuit

qcircuit = QrackCircuit()
random_circuit(6, qcircuit)
qcircuit.out_to_file('qrack_circuit.qc')

data = ''
with open('qrack_circuit.qc', 'r') as file:
    data = file.read().split()
    
print("Gate count (after optimization): " + str(data[1]))

Gate count (before optimization): 220
Gate count (after optimization): 55


In [4]:
import time

start = time.perf_counter()
qcircuit = QrackCircuit()
qcircuit.in_from_file('qrack_circuit.qc')
qsim = QrackSimulator()
qcircuit.run(qsim)
print(qsim._qubitCount)
qsim.m_all()
print("Execution seconds: " + str(time.perf_counter() - start))


Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
6
Execution seconds: 0.24648052300000245


If we have Qiskit and numpy installed, we can convert to a Qiskit circuit.

In [5]:
circ = QrackCircuit.file_to_qiskit_circuit('qrack_circuit.qc')
print(circ)

                                    ┌──────────────┐               »
q_0: ───────────────────────────────┤0             ├───────────────»
                    ┌──────────────┐│              │               »
q_1: ───────────────┤0             ├┤  Multiplexer ├───────────────»
     ┌─────────────┐│              ││              │┌─────────────┐»
q_2: ┤ Multiplexer ├┤  Multiplexer ├┤1             ├┤ Multiplexer ├»
     ├─────────────┤│              │├─────────────┬┘└─────────────┘»
q_3: ┤ Multiplexer ├┤1             ├┤ Multiplexer ├────────────────»
     ├─────────────┤└──────────────┘└─────────────┘                »
q_4: ┤ Multiplexer ├───────────────────────────────────────────────»
     ├─────────────┤                                               »
q_5: ┤ Multiplexer ├───────────────────────────────────────────────»
     └─────────────┘                                               »
c: 6/══════════════════════════════════════════════════════════════»
                                  

In [6]:
from qiskit.compiler.transpiler import transpile

basis_gates = ["u3", "cx"]
circ = transpile(circ, basis_gates=basis_gates, optimization_level=3)
print(circ)

global phase: 5.1711
      ┌───────────────────────────┐       ┌──────────────────────┐»
q_0: ─┤ U3(2.4205,-1.5708,3.1416) ├────■──┤ U3(π,-2.9717,1.7407) ├»
     ┌┴───────────────────────────┴─┐  │  └──────────────────────┘»
q_1: ┤ U3(2.4617,1.678e-08,-1.5708) ├──┼─────────────■────────────»
     └────┬───────────────────┬─────┘┌─┴─┐           │            »
q_2: ─────┤ U3(2.3005,-π/2,0) ├──────┤ X ├───────────┼────────────»
      ┌───┴───────────────────┴────┐ └───┘         ┌─┴─┐          »
q_3: ─┤ U3(0.8609,-2.2649,-2.6446) ├───────────────┤ X ├──────────»
      ├────────────────────────────┤               └───┘          »
q_4: ─┤ U3(0.72965,1.0719,-1.2415) ├──────────────────────────────»
     ┌┴────────────────────────────┴┐                             »
q_5: ┤ U3(0.74168,-1.3524,-0.16223) ├─────────────────────────────»
     └──────────────────────────────┘                             »
c: 6/═════════════════════════════════════════════════════════════»
                           

If the basis gates can be converted, a Qiskit `QuantumCircuit` can also be read into a `QrackCircuit`.

In [7]:
qcircuit = QrackCircuit()
qcircuit.in_from_qiskit_circuit(circ)
qcircuit.out_to_file('qiskit.qc')